In [22]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [24]:
SEQ_LEN = 2 * 2048  # 2 seconds at 2048 Hz
LATENT_DIM = 16
OBS_DIM = 1
LGSSM = tfd.LinearGaussianStateSpaceModel

In [29]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(1 + 1),
        tfp.layers.DistributionLambda(
            lambda t: LGSSM(
                num_timesteps=SEQ_LEN,
                transition_matrix=tf.linalg.LinearOperatorIdentity(LATENT_DIM),
                transition_noise=tfd.MultivariateNormalDiag(
                    scale_diag=(t[..., :1] ** 2) * tf.ones([LATENT_DIM])
                ),
                observation_matrix=tf.random.normal((OBS_DIM, LATENT_DIM)),
                observation_noise=tfd.MultivariateNormalDiag(
                    scale_diag=(t[..., 1:] ** 2) * tf.ones([OBS_DIM])
                ),
                initial_state_prior=tfd.MultivariateNormalDiag(
                    scale_diag=tf.ones([LATENT_DIM])
                )
            )
        ),
    ]
)

In [19]:
def negative_ll(x, dist: LGSSM) -> float:
    return -dist.forward_filter(x).log_likelihoods

In [27]:
mock_data = np.random.normal(loc=0.0, scale=1.0, size=SEQ_LEN)

In [30]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.2), loss=negative_ll)
history = model.fit(mock_data, mock_data, epochs=10, verbose=0)

ValueError: in user code:

    C:\Users\jason\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\jason\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\jason\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\jason\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\jason\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [21]:
model.trainable_variables

ValueError: Weights for model sequential_1 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.